# artsy

In [1]:
from pymongo import MongoClient
import requests

import credentials

## setup

### database

In [2]:
client = MongoClient()
# client.drop_database('artsy')
db = client.artsy

### base url

In [3]:
BASE_URL = 'https://api.artsy.net/api'

### app

In [4]:
app_name = credentials.artsy_app_name
client_id = credentials.artsy_client_id
client_secret = credentials.artsy_client_secret

### token

In [5]:
headers = {}
payload = {'client_id': client_id, 'client_secret': client_secret}
url = BASE_URL + '/tokens/xapp_token'
response = requests.post(url, params=payload, headers=headers)
token = response.json()['token']

### auxiliar

In [6]:
def change_id_field(d):
    d['_id'] = d.pop('id')

### test

In [7]:
# headers = {'X-Xapp-Token': token}
# payload = {'total_count': 1}
# url = 'https://api.artsy.net/api/artists?similar_to_artist_id=4d8b92b34eb68a1b2c0003f4'
# response = requests.get(url, params=payload, headers=headers)
# # total_count = response.json()['total_count']
# d=response.json()['_embedded']['artists']
# print(d)

## artsy resources
* Status
* Artists
* Artworks
* Editions
* Genes
* Profiles
* Partners
* Shows
* Fairs
* Applications
* Users
* User Details
* Collections
* Collection Items
* Images
* Search

## status

In [8]:
headers = {}
payload = {'total_count': 1}
url = BASE_URL + '/status'
response = requests.get(url, params=payload, headers=headers)
status = response.json()['status']
print('Status: %s' % status)

Status: good


## resources
Status, Artists, Artworks, Editions, Genes, Profiles, Partners, Shows, Fairs, Applications, Users, User Details, Collections, Collection Items, Images, Search
        artists, artworks,           genes, profiles, partners, shows, fairs

### similar_artists

In [12]:
headers = {'X-Xapp-Token': token}
payload = {}
db.similar_artists.drop()
total_count = db.artists.count()
count = 0
# try:
    size = 1000
    chunks = [db.artists.find()[x:x+size] for x in range(0, total_count, size)]
    for chunk in chunks:
        ds = []
        for artist in chunk:
            url = artist['_links']['similar_artists']['href']
            response = requests.get(url, params=payload, headers=headers)
            data = response.json()['_embedded']['artists']
            ids = [d['id'] for d in data]
            ds.append({'_id': artist['_id'], 'data': ids})
            count += 1
            if count % (total_count / 10) < len(data):
                print('%d %s processed.' % (count, 'artists'))
        db.similar_artists.insert_many(ds)
# except Exception as e:
#     print(e)
#     print(url)
#     print(response)
#     print(count)

SyntaxError: invalid syntax (<ipython-input-12-4ced32a4c507>, line 1)

### users

In [32]:
headers = {'X-Xapp-Token': token}
payload = {}
db.users.drop()
total_count = db.profiles.count()
count = 0
# try:
size = 1000
chunks = [db.profiles.find()[x:x+size] for x in range(0, total_count, size)]
for chunk in chunks:
    ds = []
    for profile in chunk:
        if 'owner' in profile['_links']:
            url = profile['_links']['owner']['href']
            url_parts = url.split('/')
            if url_parts[-2] == 'users':
                response = requests.get(url, params=payload, headers=headers)
                d = response.json()
                data = [d]
                [change_id_field(d) for d in data]
                ds.append(d)
        count += 1
        if count % (total_count / 10) < len(data):
            print('%d %s processed.' % (count, 'artists'))
    db.users.insert_many(ds)
# except Exception as e:
#     print(e)
#     print(url)
#     print(response)
#     print(count)

247 artists processed.
494 artists processed.
740 artists processed.
987 artists processed.
1233 artists processed.
1480 artists processed.
1727 artists processed.
1973 artists processed.
2220 artists processed.
2466 artists processed.


In [41]:
headers = {'X-Xapp-Token': token}
payload = {}
db.users.drop()
total_count = db.profiles.count()
c1,c2,c3,c4,c5 = 0,0,0,0,0
# try:
size = 1000
chunks = [db.profiles.find()[x:x+size] for x in range(0, total_count, size)]
for chunk in chunks:
    ds = []
    for profile in chunk:
        if 'owner' in profile['_links']:
            url = profile['_links']['owner']['href']
            url_parts = url.split('/')
            if url_parts[-2] == 'users':
                c1+=1
            elif url_parts[-2] == 'partners':
                c2+=1
            elif url_parts[-2] == 'fairs':
                c3+=1
            else:
                c4+=1
        else:
            c5 += 1
#         if count % (total_count / 10) < len(data):
#             print('%d %s processed.' % (count, 'artists'))
#     db.users.insert_many(ds)
# except Exception as e:
#     print(e)
#     print(url)
#     print(response)
#     print(count)
(c1,c2,c3,c4,c5, total_count)

(701, 1648, 107, 0, 10, 2466)

In [31]:
profile

{'_id': '54903ebb776f7215340a0000',
 '_links': {'permalink': {'href': 'http://www.artsy.net/chart-1'},
  'self': {'href': 'https://api.artsy.net/api/profiles/54903ebb776f7215340a0000'},
  'website': {'href': ''}},
 'created_at': '2014-12-16T14:16:27+00:00',
 'description': '',
 'handle': 'chart-1',
 'location': '',
 'updated_at': '2014-12-16T14:18:46+00:00'}

In [37]:
a1,a2=0

TypeError: 'int' object is not iterable

In [9]:
db.similar_artists.drop()
total_count = db.artists.count()
count = 0
try:
    for artist in db.artists.find():
        url = artist['_links']['similar_artists']['href']
        headers = {'X-Xapp-Token': token}
        payload = {}
        response = requests.get(url, params=payload, headers=headers)
        data = response.json()['_embedded']['artists']
        ids = [d['id'] for d in data]
        db['similar_artists'].insert_one({'_id': artist['_id'], 'data': data})
        count += 1
        if count % (total_count / 10) < len(data):
            print('%d %s processed.' % (count, 'artists'))
except Exception as e:
    print(e)
    print(url)
    print(response)
    print(count)

Cursor not found, cursor id: 207244522241
https://api.artsy.net/api/artists?similar_to_artist_id=515ce3415eeb1c524c001105
<Response [200]>
3329


## search

In [10]:
search_term = 'Andy Warhol'

In [11]:
search_url = BASE_URL + '/search'
headers = {'X-Xapp-Token': token}
payload = {'q': search_term}
response = requests.get(search_url, params=payload, headers=headers)

In [12]:
response

<Response [200]>

In [13]:
response.json()

{'_embedded': {'results': [{'_links': {'permalink': {'href': 'https://www.artsy.net/artist/andy-warhol'},
     'self': {'href': 'https://api.artsy.net/api/artists/andy-warhol'},
     'thumbnail': {'href': 'https://d32dm0rphc51dk.cloudfront.net/PFufT6nMKNwLOpPEezf4Ww/large.jpg'}},
    'description': 'Dec 5, 2015 ... Browse the best of Andy Warhol art, including paintings and artwork for sale (with \nprices), upcoming shows, and exclusive Andy Warhol articles\xa0...',
    'og_type': 'artsyinc:artist',
    'title': 'Andy Warhol Art - 1161 Artworks, Biography & Shows on Artsy',
    'type': 'Artist'},
   {'_links': {'permalink': {'href': 'https://www.artsy.net/artist/andy-warhol/works'},
     'self': {'href': 'https://api.artsy.net/api/artists/andy-warhol'},
     'thumbnail': {'href': 'https://i.embed.ly/1/display/resize?height=300&quality=95&grow=false&url=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FVFelegc1J0GFxkS2LSJdRg%2Flarger.jpg&key=a1f82558d8134f6cbebceb9e67d04980'}},
    'descrip